In [ ]:
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import csv
import os
import numpy as np
import pandas as pd

import ast

import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import re


import warnings 
warnings.filterwarnings("ignore")

# SCRAPING ANNUAIRE EXPERT COMPTABLE

In [ ]:
i_progress_file_path = 'progress_comptable_i3.txt'
d_progress_file_path = 'progress_comptable_d3.txt'
csv_file_path = 'comptable2_3.csv'

In [ ]:

def read_progress(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            line = f.readline()
            if line:
                return int(line.strip())
    return 1  # Démarre de 1 si le fichier n'existe pas ou est vide

def save_progress(file_path, progress):
    with open(file_path, 'w') as f:
        f.write(str(progress))


columns = ['company_name', 'forme_juridique', 'telephone', 'nb_exp', 'sites']
# creation du fichier csv si il n'existe pas déjà du à une progression antérieure
if not os.path.exists(csv_file_path):
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=columns)
        writer.writeheader()

i_cookie = 55#read_progress(i_progress_file_path)
# ouvrir l'annuaire des experts comptables
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# see progression with tqdm starting from the last page
max_i = 55
# read_progress(i_progress_file_path
for i in tqdm(range(55, max_i+1)):
    driver.get(f"https://annuaire.experts-comptables.org/recherche/{i}?seed=33798")
    if i == i_cookie:
        cookie_accept = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="tarteaucitronPersonalize"]')))[0]
        cookie_accept.click()
        driver.maximize_window()
    # read_progress(d_progress_file_path)
    for d in range(1 , 11):
        try:
            base_xpath = f"/html/body/div[1]/div[2]/div/div[2]/div[{d}]"
            societe = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f"{base_xpath}/div[2]/div[1]/div/a"))).text
            company_name, forme_juridique = societe.split('\n')[:2]
            try:
                telephone_button = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, f"{base_xpath}/div[3]/div/div[1]")))
                telephone_button.click()
                try:
                    telephone = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="mailForm"]/div/a'))).text
                except:
                    telephone = np.nan
                close_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-close"]')))
                close_button.click()
            except: 
                telephone = np.nan
            try : 
                savoir_plus_button =  WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, f'{base_xpath}/div[3]/a')))
                savoir_plus_button.click()
                try :
                    nb_exp = len(WebDriverWait(driver, 1).until(EC.presence_of_all_elements_located((By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[2]/div[1]/div/div/div'))))
                except : 
                    nb_exp = np.nan
                try :
                    element = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[2]/div[2]/div[2]/div/p")))
                    sites = element.text
                except:
                    sites = np.nan
                try :
                    bouton_retour = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[1]/div[1]/div/span[2]/a')))
                    bouton_retour.click()
                except :
                    print(f'pas de boutton retour error i: {i}, d:{d}')
                    sys.quit(1)
            except : 
                print(f'pas de boutton en savoir plus error i: {i}, d:{d}')
                sys.quit(1)                
        except:
            print(f'societe non trouvé error i: {i}, d:{d}')
            continue    


        # stockage des données enregistrées
        data = {
            'company_name': company_name,
            'forme_juridique': forme_juridique,
            'telephone': telephone,
            'nb_exp': nb_exp,
            'sites' : sites
        }

        with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=columns)
            writer.writerow(data)

        if d == 10:
            save_progress(d_progress_file_path, 1)
        else:
            save_progress(d_progress_file_path, d)

    
    save_progress(i_progress_file_path, i)
    


# SCRAPING PAPPERS

In [ ]:
progress_file_path = 'progress_papers_test.txt'
csv_file_path = 'pappers_comptable_test.csv'
liste_file_path = 'liste_pappers.txt'

In [ ]:
def convert_to_float(s):
    if s == '' or s == np.nan:
        return np.nan
    if not isinstance(s, str):  # Ensure chaine is a string
        return s
    # Replace commas with periods to handle European decimal format and remove spaces
    s = s.replace(',', '.').replace(' ', '')
    multipliers = {'K': 1e3, 'M': 1e6, 'B': 1e9}
    # Identify if the last character is a multiplier
    last_char = s[-1].upper()  # Use upper to handle lowercase letters
    if last_char in multipliers:
        # Convert the numeric part to float and apply the multiplier
        return float(s[:-1]) * multipliers[last_char]
    else:
        # If there's no multiplier, directly convert to float
        return float(s)

In [ ]:
def read_progress(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            line = f.readline()
            if line:
                return int(line.strip()) + 1
    return 0  

In [ ]:
def save_progress(file_path, progress):
    with open(file_path, 'w') as f:
        f.write(str(progress))

In [ ]:

with open(liste_file_path, 'r') as file:
    content = file.read()
liste = content.split('\n')
for i in range(len(liste)):
    liste[i] = liste[i].replace("'",'')

In [ ]:
token = "d25b6f1b63f9439193fd13f95a21d97421ba8670669"
proxyModeUrl = "http://{}:@proxy.scrape.do:8080".format(token)
proxies = {
    "http": proxyModeUrl,
    "https": proxyModeUrl,
}

In [ ]:
columns = ['nom_beneficiaire','qualite','age','date_de_naissance','company_name', 'siren', 'adresse', 'effectif', 'annee_performance', 'CA', 'resultat_net', 'ebitda', 'papers_link']
# creation du fichier csv si il n'existe pas déjà du à une progression antérieure

if not os.path.exists(csv_file_path):
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=columns)
        writer.writeheader()

start_from = read_progress(progress_file_path)

def session_requests(prox, verify=False):
    session = requests.Session()
    session.proxies = prox
    session.verify = verify
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session



def scrape_data(url, session):
    response = session.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        actif_span = soup.find('span', class_='actif')
        if actif_span:  # Vérifie si `actif_span` n'est pas None
            status_text = actif_span.get_text(strip=True)
            if status_text != 'Active':
                return
        else:
            return
        
        
        # Siren extraction
        siren = url.split('-')[-1] if url else np.nan

        # Adresse extraction
        adresse_element = soup.find("th", text="Adresse :")
        adresse = adresse_element.find_next_sibling("td").text.strip() if adresse_element and adresse_element.find_next_sibling("td") else np.nan

        # Effectif extraction
        effectif_element = soup.find("th", text="Effectif :")
        effectif = effectif_element.find_next_sibling("td").text.strip().split('\n')[0] if effectif_element and effectif_element.find_next_sibling("td") else np.nan

        # Année de performance extraction
        annee_performance_element = soup.find("th", text="Performance")
        annee_performance = annee_performance_element.find_next_sibling("th").text.strip() if annee_performance_element and annee_performance_element.find_next_sibling("th") else np.nan

        # Chiffre d'affaires extraction
        ca_element = soup.find("th", text=re.compile("Chiffre d'affaires"))
        ca = ca_element.parent.find("td").text.strip() if ca_element and ca_element.parent and ca_element.parent.find("td") else np.nan
        ca = convert_to_float(ca) if ca is not np.nan else np.nan

        # Résultat net extraction
        resultat_net_element = soup.find("th", text="Résultat net (€)")
        resultat_net = resultat_net_element.find_next_sibling("td").text.strip() if resultat_net_element and resultat_net_element.find_next_sibling("td") else np.nan
        resultat_net = convert_to_float(resultat_net) if resultat_net is not np.nan else np.nan

        # EBITDA extraction
        ebitda_element = soup.find("th", text="EBITDA - EBE (€)")
        ebitda = ebitda_element.find_next_sibling("td").text.strip() if ebitda_element and ebitda_element.find_next_sibling("td") else np.nan
        ebitda = convert_to_float(ebitda) if ebitda is not np.nan else np.nan

        company_name = ' '.join([part.upper() for part in url.split('/')[-1].split('-') if not part.isdigit() and part])


        if ebitda is np.nan and resultat_net is not np.nan:
            ebitda = resultat_net * 1.25

        for li in soup.select('.beneficiaire'):
        # Nom du bénéficiaire
            nom_element = li.find('a', class_='nom')
            nom = nom_element.text.strip() if nom_element else np.nan
            
            # Qualité du bénéficiaire
            qualite_element = li.find('span', class_='qualite')
            qualite = qualite_element.text.strip() if qualite_element else np.nan
            
            # Âge et date de naissance du bénéficiaire
            age_element = li.find('span', class_='age')
            if age_element:
                age_details = age_element.text.strip().split('-')
                age = age_details[0].strip() if age_details else np.nan
                date_de_naissance = age_details[-1].strip() if len(age_details) > 1 else np.nan
            else:
                age = np.nan
                date_de_naissance = np.nan

            data = {
                'nom_beneficiaire': nom,
                'qualite': qualite,
                'age': age,
                'date_de_naissance': date_de_naissance,
                'company_name': company_name,  
                'siren': siren,
                'adresse': adresse,
                'effectif': effectif,
                'annee_performance': annee_performance,
                'CA': ca,
                'resultat_net': resultat_net,
                'ebitda': ebitda,
                'papers_link': url  
            }

            return data
    else:
        return 

def main(liste, start_from, proxies, csv_file_path, progress_file_path, columns):
    print(f"Starting scraping from index {start_from}")
    liste_url = []
    for i in range(len(liste)):
        url = f"https://www.pappers.fr/entreprise/{liste[i]}"
        liste_url.append(url)
    session = session_requests(proxies)
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(scrape_data, s, session): s for s in liste_url[start_from:]}
        for i, future in enumerate(tqdm(concurrent.futures.as_completed(futures), total=len(liste_url[start_from:]), desc="Progression du scraping")):
            s = futures[future]
            try:
                result = future.result()
                if result:
                    with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:
                        writer = csv.DictWriter(csvfile, fieldnames=columns)
                        writer.writerow(result)
            
            except Exception as e:
                print(f"An error occurred for {s}: {str(e)}")
            finally:
                save_progress(progress_file_path, i)

if __name__ == "__main__":
    main(liste, start_from, proxies, csv_file_path, progress_file_path, columns)
   

# SCRAPING LINKEDIN COMPANY

In [ ]:
file_path = 'liste_linkedin_entreprise.txt'
csv_file_path = 'entreprise_linkedin.csv'
progress_file_path = 'progress_linkedin_entreprise.txt'

In [29]:
def read_progress(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            line = f.readline()
            if line:
                return int(line.strip()) + 1
    return 0  

In [30]:
def save_progress(file_path, progress):
    with open(file_path, 'w') as f:
        f.write(str(progress))

In [31]:

with open(file_path, 'r') as file:
    content = file.read()

liste = content.split('\n')

In [32]:
liste_linkedin = []
for i in range(len(liste)):
    liste_linkedin.append(liste[i].replace(' ','%20'))

In [34]:
def read_progress(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            line = f.readline()
            if line:
                return int(line.strip()) + 1
    return 0  

In [35]:
def save_progress(file_path, progress):
    with open(file_path, 'w') as f:
        f.write(str(progress))

In [136]:
columns = ['company_name','linkedin_company_url', 'site_web', 'taille']
if not os.path.exists(csv_file_path):
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=columns)
        writer.writeheader()


driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# see progression with tqdm starting from the last page
driver.get(f"https://www.linkedin.com/search")
cookie_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]')))
cookie_button.click()

mail = 'email'
password = 'password'  

login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="username"]')))
password_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="password"]')))
connect_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="organic-div"]/form/div[3]/button')))

login_button.send_keys(mail)
password_button.send_keys(password)
connect_button.click()

start_from = read_progress(progress_file_path)

for i, company in enumerate(tqdm(liste_linkedin[start_from:]), start=start_from):

    driver.get(f"https://www.linkedin.com/search/results/companies/?companyHqGeo=%5B%22105015875%22%5D&industryCompanyVertical=%5B%2247%22%5D&keywords={company}&origin=FACETED_SEARCH&sid=%3AbD")
    try : 
        first_link_company_button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="search-results-container"]/div[2]/div/ul/li[1]/div//*[@class="display-flex"]')))
        first_link_company_button.click()
        try:
            a_propos_button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//li[contains(@class, 'org-page-navigation__item') and .//text()='À propos']//a")))
            a_propos_button.click()
            linkedin_company_url = driver.current_url
            a_propos_element = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.XPATH, "//*[@class = 'overflow-hidden']")))
            a_propos = a_propos_element.text.split('\n')
            try :
                index = a_propos.index('Site web')
                site_web = a_propos[index+1]
            except:
                site_web = np.nan
            try:
                index = a_propos.index('Taille de l’entreprise')
                taille = a_propos[index+1]
            except:
                taille = np.nan

            data = {'linkedin_company_url':linkedin_company_url,
                    'site_web':site_web,
                    'taille':taille,
                    'company_name': liste[i]}

        except:
                data = {'linkedin_company_url':linkedin_company_url,
                    'site_web':np.nan,
                    'taille':np.nan,
                    'company_name': liste[i]}
    except:
        data = {'linkedin_company_url':np.nan,
                'site_web':np.nan,
                'taille':np.nan,
                'company_name': liste[i]}
          

    with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:
                        writer = csv.DictWriter(csvfile, fieldnames=columns)
                        writer.writerow(data)
                        
    save_progress(progress_file_path, i)
         

100%|██████████| 449/449 [25:13<00:00,  3.37s/it]


# SCRAPING LINKEDIN PROFIL

In [63]:
csv_file_path = 'profil_linkedin.csv'
progress_file_path = 'progress_linkedin_profil.txt'

In [64]:
file_path = 'rocket_name_company.csv'
rocket_df = pd.read_csv(file_path)

In [65]:
rocket_df['liste'] = rocket_df['Name'] + ', ' + rocket_df['Current Employer']

In [66]:
liste_linkedin = rocket_df['liste'].tolist()

In [67]:
for i in range (len(liste_linkedin)):
    liste_linkedin[i] = liste_linkedin[i].replace(' ','%20').replace(',','%2C')

In [104]:
columns = ['Name','company_name','linkedin_profil_url']
if not os.path.exists(csv_file_path):
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=columns)
        writer.writeheader()


driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# see progression with tqdm starting from the last page
driver.get(f"https://www.linkedin.com/search")
cookie_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]')))
cookie_button.click()

mail = 'romain_foucault@icloud.com'
password = '250768Lebigboss'  

login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="username"]')))
password_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="password"]')))
connect_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="organic-div"]/form/div[3]/button')))

login_button.send_keys(mail)
password_button.send_keys(password)
connect_button.click()

start_from = read_progress(progress_file_path)

for i, profil in enumerate(tqdm(liste_linkedin[start_from:]), start=start_from):

    driver.get(f"https://www.linkedin.com/search/results/people/?keywords={profil}&origin=SWITCH_SEARCH_VERTICAL&sid=rxf")
    try : 
        first_link_profil_button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, '(//a[@class="app-aware-link "])[2]')))
        linkedin_profil_url = first_link_profil_button.get_attribute('href')
    except:
        linkedin_profil_url = np.nan

    data = {'linkedin_profil_url':linkedin_profil_url,
            'Name':rocket_df.loc[i,'Name'],
            'company_name': rocket_df.loc[i,'Current Employer']}
          

    with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:
                        writer = csv.DictWriter(csvfile, fieldnames=columns)
                        writer.writerow(data)
                        
    save_progress(progress_file_path, i)
         

100%|██████████| 1955/1955 [1:26:03<00:00,  2.64s/it]
